###  Testing with SDK v2

In [ ]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.identity import DefaultAzureCredential
import sys

# Enter details of your AML workspace
subscription_id = "<YOUR_SUBSCRIPTION_ID>"
resource_group = "<YOUR_RESOURCE_GROUP>"
workspace = "<YOUR_WORKSPACE_NAME>"
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

sys.path.append("../") # go to parent dir

In [ ]:
from ray_on_aml.core import Ray_On_AML


In [ ]:
pip uninstall ray_on_aml --y

In [ ]:
pip install git+https://github.com/microsoft/ray-on-aml.git@james-dev

In [ ]:
# from src.ray_on_aml.core import Ray_On_AML

import logging
ray_on_aml =Ray_On_AML(ml_client=ml_client, compute_cluster ="d13")
#Note that if you need to customize the pip installation of the cluster, you also needs to support the ray package e.g. ray[data] which 
#match the version of the ray package(s) in your compute instance. If you don't specify pip_packages then ray[default] is inserted 
#automatically

# For use as client mode, uncomment these lines
ray = ray_on_aml.getRay(num_node=2,pip_packages=["ray[air]==2.1.0","ray[data]==2.1.0","torch==1.13.0","fastparquet==2022.12.0", "azureml-mlflow==1.48.0", "pyarrow==6.0.1", "dask==2022.2.0", "adlfs==2022.11.2", "fsspec==2022.11.0"])
client = ray.init(f"ray://{ray_on_aml.headnode_private_ip}:10001")

#use CI as head node

# ray = ray_on_aml.getRay(ci_is_head=True, num_node=2)

In [2]:
#Example of using input and output for interactive job
from azure.ai.ml import command, Input, Output
from src.ray_on_aml.core import Ray_On_AML
import logging
ray_on_aml =Ray_On_AML(ml_client=ml_client, compute_cluster ="ds11", verbosity=logging.INFO )

inputs={

    "ISDWeather": Input(
        type="uri_folder",
        path="azureml://datastores/adlsstore0001/paths/ISDWeather/year=2008",
    )
}

outputs={
    "output1": Output(
        type="uri_folder",
        path="azureml://datastores/adlsstore0001/paths/dev",
    ),
    "output2": Output(
        type="uri_folder",
        path="azureml://datastores/adlsstore0001/paths/dev",
    )
}

ray = ray_on_aml.getRay(inputs = inputs,outputs=outputs, num_node=2,pip_packages=["ray[data]", "fastparquet", "azureml-mlflow", "pyarrow==6.0.1", "dask==2022.2.0", "adlfs", "fsspec"])

Waiting cluster to start and return head node's ip
..............................................................
 cluster is ready, head node ip  10.0.0.11


In [ ]:

ray.cluster_resources()

In [ ]:
ray_on_aml.shutdown()

In [ ]:
from adlfs import AzureBlobFileSystem
#install dask, adlfs 
import dask.dataframe as dd
from ray.util.dask import ray_dask_get, enable_dask_on_ray, disable_dask_on_ray
enable_dask_on_ray()
abfs = AzureBlobFileSystem(account_name="azureopendatastorage",  container_name="isdweatherdatacontainer")

storage_options = {'account_name': 'azureopendatastorage'}
ddf = dd.read_parquet('az://nyctlc/green/puYear=2019/puMonth=*/*.parquet', storage_options=storage_options)

# data = ray.data.read_parquet(ray_on_aml.mount_points['ISDWeather'])

data = ray.data.read_parquet("az://isdweatherdatacontainer/ISDWeather/year=2009", filesystem=abfs)
print(data.count())

In [ ]:
client.disconnect()

###  Testing with SDK v1

In [ ]:
pip install --upgrade Ray_On_AML

In [1]:
from azureml.core import Workspace
import sys
# sys.path.append("../") # go to parent dir
import importlib


In [3]:
from ray_on_aml.core import Ray_On_AML
ws = Workspace.from_config()
ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="d13",maxnode=2)
ray = ray_on_aml.getRay(ci_is_head =True, num_node=2,pip_packages=["ray[air]==2.2.0","ray[data]==2.2.0","torch==1.13.0","fastparquet==2022.12.0", 
"azureml-mlflow==1.48.0", "pyarrow==6.0.1", "dask==2022.12.0", "adlfs==2022.11.2", "fsspec==2022.11.0"])


Usage stats collection will be enabled by default in the next release. See https://github.com/ray-project/ray/issues/20857 for more details.
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38/bin//ray", line 8, in <module>
    sys.exit(main())
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/scripts/scripts.py", line 2269, in main
    return cli()
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/click/core.py", line 1055, in main
    rv = self.invoke(ctx)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/click/core.py", line 1657, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/click/core.py", line 1404, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/anaconda/envs/azureml_p

Waiting for cluster to start

 Cluster started successfully


In [15]:
ray.cluster_resources()

{'CPU': 8.0,
 'memory': 32975051982.0,
 'object_store_memory': 16487525990.0,
 'node:10.0.0.7': 1.0}

(raylet, ip=10.0.0.12) [2022-12-15 17:05:48,104 C 81 81] (raylet) node_manager.cc:1060: [Timeout] Exiting because this node manager has mistakenly been marked as dead by the GCS: GCS didn't receive heartbeats from this node for 30000 ms. This is likely because the machine or raylet has become overloaded.
(raylet, ip=10.0.0.12) *** StackTrace Information ***
(raylet, ip=10.0.0.12) /azureml-envs/azureml_42918adb77bf7a3950b0829265adb0c8/lib/python3.8/site-packages/ray/core/src/ray/raylet/raylet(+0x4fa17a) [0x558d478e617a] ray::operator<<()
(raylet, ip=10.0.0.12) /azureml-envs/azureml_42918adb77bf7a3950b0829265adb0c8/lib/python3.8/site-packages/ray/core/src/ray/raylet/raylet(+0x4fbc52) [0x558d478e7c52] ray::SpdLogMessage::Flush()
(raylet, ip=10.0.0.12) /azureml-envs/azureml_42918adb77bf7a3950b0829265adb0c8/lib/python3.8/site-packages/ray/core/src/ray/raylet/raylet(+0x4fbf67) [0x558d478e7f67] ray::RayLog::~RayLog()
(raylet, ip=10.0.0.12) /azureml-envs/azureml_42918adb77bf7a3950b0829265adb0c

In [ ]:
pip install aiohttp

In [24]:
import ray

# Load data.
dataset = ray.data.read_csv("s3://anonymous@air-example-data/breast_cancer.csv")

# Split data into train and validation.
train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)

# Create a test dataset by dropping the target column.
test_dataset = valid_dataset.drop_columns(cols=["target"])

RayTaskError(ImportError): [36mray::_get_read_tasks()[39m (pid=20575, ip=10.0.0.7)
  File "/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/ray/data/read_api.py", line 1380, in _get_read_tasks
    reader = ds.create_reader(**kwargs)
  File "/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/ray/data/datasource/file_based_datasource.py", line 219, in create_reader
    return _FileBasedDatasourceReader(self, **kwargs)
  File "/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/ray/data/datasource/file_based_datasource.py", line 369, in __init__
    _check_pyarrow_version()
  File "/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/ray/data/_internal/util.py", line 78, in _check_pyarrow_version
    raise ImportError(
ImportError: Datasets requires pyarrow >= 6.0.1, < 7.0.0, but 9.0.0 is installed. Reinstall with `pip install -U "pyarrow<7.0.0"`. If you want to disable this pyarrow version check, set the environment variable RAY_DISABLE_PYARRO

In [25]:
!pip install pyarrow==6.0.1

(raylet, ip=10.0.0.8) [2022-12-15 16:36:11,596 C 92 92] (raylet) node_manager.cc:1060: [Timeout] Exiting because this node manager has mistakenly been marked as dead by the GCS: GCS didn't receive heartbeats from this node for 30000 ms. This is likely because the machine or raylet has become overloaded.
(raylet, ip=10.0.0.8) *** StackTrace Information ***
(raylet, ip=10.0.0.8) /azureml-envs/azureml_42918adb77bf7a3950b0829265adb0c8/lib/python3.8/site-packages/ray/core/src/ray/raylet/raylet(+0x4fa17a) [0x55f58e7ad17a] ray::operator<<()
(raylet, ip=10.0.0.8) /azureml-envs/azureml_42918adb77bf7a3950b0829265adb0c8/lib/python3.8/site-packages/ray/core/src/ray/raylet/raylet(+0x4fbc52) [0x55f58e7aec52] ray::SpdLogMessage::Flush()
(raylet, ip=10.0.0.8) /azureml-envs/azureml_42918adb77bf7a3950b0829265adb0c8/lib/python3.8/site-packages/ray/core/src/ray/raylet/raylet(+0x4fbf67) [0x55f58e7aef67] ray::RayLog::~RayLog()
(raylet, ip=10.0.0.8) /azureml-envs/azureml_42918adb77bf7a3950b0829265adb0c8/lib/

In [ ]:
import torch
import torch.nn as nn
from torch.nn.modules.utils import consume_prefix_in_state_dict_if_present

from ray import train
from ray.air import session
from ray.air.config import ScalingConfig
from ray.train.torch import TorchCheckpoint, TorchTrainer


def create_model(input_features):
    return nn.Sequential(
        nn.Linear(in_features=input_features, out_features=16),
        nn.ReLU(),
        nn.Linear(16, 16),
        nn.ReLU(),
        nn.Linear(16, 1),
        nn.Sigmoid(),
    )


def train_loop_per_worker(config):
    batch_size = config["batch_size"]
    lr = config["lr"]
    epochs = config["num_epochs"]
    num_features = config["num_features"]

    # Get the Ray Dataset shard for this data parallel worker,
    # and convert it to a PyTorch Dataset.
    train_data = train.get_dataset_shard("train")
    # Create model.
    model = create_model(num_features)
    model = train.torch.prepare_model(model)

    loss_fn = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    for cur_epoch in range(epochs):
        for batch in train_data.iter_torch_batches(
            batch_size=batch_size, dtypes=torch.float32
        ):
            # "concat_out" is the output column of the Concatenator.
            inputs, labels = batch["concat_out"], batch["target"]
            optimizer.zero_grad()
            predictions = model(inputs)
            train_loss = loss_fn(predictions, labels.unsqueeze(1))
            train_loss.backward()
            optimizer.step()
        loss = train_loss.item()
        session.report({"loss": loss}, checkpoint=TorchCheckpoint.from_model(model))


num_features = len(train_dataset.schema().names) - 1

trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config={
        "batch_size": 128,
        "num_epochs": 20,
        "num_features": num_features,
        "lr": 0.001,
    },
    scaling_config=ScalingConfig(
        num_workers=2,  # Number of workers to use for data parallelism.
        use_gpu=False,
        trainer_resources={"CPU": 1},  # so that the example works on Colab.
    ),
    datasets={"train": train_dataset},
    preprocessor=preprocessor,
)
# Execute training.
result = trainer.fit()
print(f"Last result: {result.metrics}")
# Last result: {'loss': 0.6559339960416158, ...}

In [ ]:
pip install --upgrade torch